In [ ]:
# fmt: off
%matplotlib inline
%config InlineBackend.figure_format = "retina"
%load_ext autoreload
%load_ext jupyter_black
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mls_scf_tools.mls_pint import ureg
plt.style.use("mls_scf")
def vdir(obj):
    return [x for x in dir(obj) if not x.startswith('__')]


In [ ]:
import openpyxl

In [ ]:
# filename = (
#     "/Users/livesey/Library/CloudStorage/"
#     "OneDrive-SharedLibraries-TheNationalAcademies/"
#     "Jones, Christopher - Views WRC-27/"
#     "Views_response to review/Views WRC-27 - Report Comments.xlsx"
# )
filename = "~/Desktop/text.xlsx"

In [ ]:
section_map = {
    0: "Front matter",
    239: "Preface",
    308: "Executive summary",
    342: "Introduction",
    700: "WRC-27 opening",
    750: "AI-1.1",
    914: "AI-1.2",
    1000: "AI-1.3",
    1118: "AI-1.4",
    1179: "AI-1.5",
    1190: "AI-1.6",
    1224: "AI-1.7",
    1364: "AI-1.8",
    1591: "AI-1.10",
    1713: "AI-1.11",
    1798: "AI-1.12",
    1927: "AI-1.13",
    2165: "AI-1.14",
    2170: "AI-1.15",
    2353: "AI-1.16",
    2562: "AI-1.17",
    2658: "AI-1.18",
    2853: "AI-1.19",
    2900: "WRC-31 opening",
    2921: "PAI-2.1 & 2.6",
    3045: "PAI-2.2",
    3110: "PAI-2.3",
    3212: "PAI-2.4",
    3291: "PAI-2.9",
    3431: "PAI-2.10 & 2.11",
    3678: "PAI-2.12 & 2.13",
    3822: "PAU-2.14",
    3879: "Conclusions",
    3916: "Appendix A",
    3944: "Appendix B",
    4351: "Appendix C",
    4485: "Appendix D",
}
line_number_transitions = np.array(list(section_map.keys()), dtype=int) - 1

In [ ]:
def get_section(line_number: int) -> str:
    """Workout which section a line is in"""
    i_section = np.searchsorted(line_number_transitions, line_number)
    key = int(line_number_transitions[max(i_section - 1, 0)]) + 1
    return section_map[key]

In [ ]:
workbook = openpyxl.load_workbook(filename=filename)
worksheet = workbook.worksheets[0]

In [ ]:
column = worksheet["D5":"D235"]
results = list()
for cell_tuple in column:
    reference = str(cell_tuple[0].value)
    clauses = [clause.strip() for clause in reference.split(",")]
    result = set()
    for clause in clauses:
        # Skip clauses that start with certain prefixes
        if any(clause.startswith(prefix) for prefix in ["Figure", "Table", "Pg"]):
            continue
        # Strip off any leading "Line "s
        if clause.startswith("Line "):
            clause = clause[5:]
        if clause == "General":
            result.add(clause)
        elif clause == "None":
            result.add("")
        else:
            line_numbers = [int(word) for word in clause.split("-")]
            result |= set([get_section(line_number) for line_number in line_numbers])
    results.append(", ".join(list(result)))

In [ ]:
# Now insert the results into the spreadsheet
for row, entry in zip(range(5, 236), results):
    worksheet[f"I{row}"] = entry

In [ ]:
# help(workbook.save)
workbook.save("/Users/livesey/Desktop/test.xlsx")